In [1]:
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.decomposition import TruncatedSVD # PCA used for sparce matrices
from sklearn.cluster import KMeans # used for clustering
from sklearn.pipeline import make_pipeline #developing a pipeline for our fit transform
import pandas as pd

In [2]:
def book_importer(file_name): # to import the books as a text file
    with open(file_name, 'r', encoding="utf8") as myfile:
        line = myfile.read().replace('\n', ' ').replace(",","").replace(":","").replace("ۀ","ه").replace("-","").replace("،","")
        line = re.sub("\d","",line) # removing numbers
    return line

In [3]:
# Importing all the poet books into a list
pattern = 'poets/*.txt'
poet_files = glob.glob(pattern)

In [4]:
poet_labels_persian = ['ابوسعید ابوالخیر', 'امیر معظی'
                       , 'انوری', 'عارف قزوینی', 'اسعد گرگانی', 'اسدی توسی', 'عطار نیشابوری',
 'ارزقی', 'بابافاضل', 'باباطاهر', 'شیخ بهایی', 'ملک الشعرای بهار', 'بیدل دهلوی', 'ابن حسام',
 'عراقی', 'فرخی',  'فردوسی', 'فیض کاشانی', 'فروغی', 'قاآنی', 'گیلانی', 'حافظ', 'هاتف اصفهانی',
 'هلالی',  'اقبال لاهوری', 'جبلی', 'جامی', 'کمال اسماعیل',  'خاقانی', 'خاجوی کرمانی',
  'خیام', 'امیرخسرو', 'مهستی', 'منوچهری', 'مسعود سعد', 'میبدی', 'محتشم',
 'مولوی', 'ناصر خسرو', 'نزاری قهستانی', 'عبید زاکانی', 'عمان سامانی', 'عنصری', 'عرفی',
 'اوحدی', 'پروین اعتصامی', 'رهی معیری', 'آرتیمانی', 'رودکی', 'سعدی', 'صائب', 'سلمان', 'سنایی',
 'سیف فرغانی', 'شبستری', 'شاه نعمت الله', 'شهریار', 'عباس صبوحی', 'وحشی بافقی',  'وطواط',
 'ظهیر']

In [5]:
documents = []
for poet in poet_files:
    documents.append(book_importer(poet)[5:])

In [6]:
tfidf = TfidfVectorizer()
csr_mat = tfidf.fit_transform(documents)

In [16]:
#developing a pipeline
svd = TruncatedSVD(n_components=5)
kmeans = KMeans(n_clusters=3)
pipeline = make_pipeline(svd, kmeans)

In [17]:
pipeline.fit(csr_mat)
labels = pipeline.predict(csr_mat) #Generating the labels for each class

In [18]:
df = pd.DataFrame({'labels': labels, 'poets': poet_labels_persian})
df= df.sort_values(by='labels')

In [19]:
for i in range(df['labels'].nunique()):
    print(df[df['labels']==i])

    labels          poets
59       0          وطواط
23       0          هلالی
24       0   اقبال لاهوری
17       0     فیض کاشانی
27       0   کمال اسماعیل
53       0     سیف فرغانی
29       0   خاجوی کرمانی
20       0         گیلانی
9        0        بباطاهر
41       0    عمان سامانی
7        0          ارزقی
6        0  عطار نیشابوری
35       0          میبدی
45       0  پروین اعتصامی
37       0          مولوی
    labels        poets
48       1        رودکی
34       1    مسعود سعد
26       1         جامی
25       1         جبلی
38       1    ناصر خسرو
33       1      منوچهری
60       1         ظهیر
42       1        عنصری
19       1        قاآنی
16       1       فردوسی
15       1         فرخی
52       1        سنایی
1        1    امیر معظی
2        1        انوری
4        1  اسعد گرگانی
5        1    اسدی توسی
    labels             poets
43       2              عرفی
44       2             اوحدی
58       2        وحشی بافقی
46       2         رهی معیری
49       2              سعدی
47